In [103]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import scale

In this Jupyter Notebook a Linear Algebra interpretaion of [PageRank](https://en.wikipedia.org/wiki/PageRank) will be used to determine the density of population of Ukrainians in Poland. The underlying intuition behind this takes it's roots from the fact that one could not know where season workers / migrants would settle in Poland, but the business and market should handle this. I would build a network of cities/districts and train a Page Rank alghorithm, trying to converge the power method to find out the rank of each city in Poland and Ukraine that is linked to a network via bus routes. Ukrainian cities/districts would be removed from the list so ranks would be calculated for a subnetwork of Polish cities/regions.

In [57]:
stops = pd.read_csv('poland_bus_routes_geocoded.csv')
stops.head()

,Unnamed: 0,id,rejs_number,stop_city_name,stop_address,stop_code,route_frequency,distance_direct,arrival_direct,departure_direct,...,arrival_return,departure_return,time_diff_return,stop_number,full_address,lat,lng,country,region,city
0,0,PL944,1,Жовква,пл. Коновальця 1,4622710100,Щоденно,0,4:20,4:40,...,14:55,NaN,0,0.0,"Жовква, пл. Коновальця 1",50.056643,23.972478,Ukraine,Lvivska oblast,Zhovkivskyi district
1,1,PL944,1,Рава Руська,"вул. Двірцева, 4",4622710400,Щоденно,32,5:25,5:30,...,14:05,14:10,0,1.0,"Рава Руська, вул. Двірцева, 4",50.230071,23.636844,Ukraine,Lvivska oblast,Zhovkivskyi district
2,2,PL944,1,Томашів Любельський,"ul. Zamojska, 9",22-600,Щоденно,63,6:05,NaN,...,11:00,11:30,0,4.0,"Томашів Любельський, ul. Zamojska, 9",50.454665,23.419679,Poland,lubelskie,tomaszowski
3,3,PL944,2,Жовква,пл. Коновальця 1,4622710100,Щоденно,0,21:50,22:10,...,6:25,NaN,0,5.0,"Жовква, пл. Коновальця 1",50.056643,23.972478,Ukraine,Lvivska oblast,Zhovkivskyi district
4,4,PL944,2,Рава Руська,"вул. Двірцева, 4",4622710400,Щоденно,32,22:55,23:00,...,5:35,5:40,0,6.0,"Рава Руська, вул. Двірцева, 4",50.230071,23.636844,Ukraine,Lvivska oblast,Zhovkivskyi district


In [58]:
# Remove Ukrainian cities from the list
stops = stops.loc[stops['country'] == 'Poland']

In [59]:
stops['city'] = stops['city'].astype('category')

In [60]:
stops['city_cat'] = stops['city'].cat.codes

In [61]:
connections = []

In [62]:
for route in tqdm(np.unique(stops.id)):
    route_direction = stops.loc[stops['id'] == route].sort_values(by=['stop_number'])
    ind = 0
    while ind < route_direction.shape[0] - 1:
        connections.append([route_direction.iloc[ind].city_cat, route_direction.iloc[ind + 1].city_cat])
        ind += 1
        

100%|██████████| 477/477 [00:03<00:00, 152.41it/s]


In [63]:
print(connections[:10])

[[8, 42], [8, 8], [8, 8], [8, 8], [51, 85], [85, 97], [97, 192], [79, 79], [79, 79], [79, 79]]


In [64]:
connections = pd.DataFrame(connections, columns = ['start_code', 'stop_code']) 

In [65]:
connections.head(20)

,start_code,stop_code
0,8,42
1,8,8
2,8,8
3,8,8
4,51,85
5,85,97
6,97,192
7,79,79
8,79,79
9,79,79


In [66]:
max_city_category = max(max(connections.start_code), max(connections.stop_code))
max_city_category

201

Hereunder a matrix of connections would be builded. Each node connection would be constructed as the number of incoming connections divided by number of outgoing connections.

In [67]:
links_matrix = np.zeros((max_city_category + 1, max_city_category + 1))

In [68]:
for stop in tqdm(np.unique(stops['city_cat'])):
    incoming = connections.loc[connections['stop_code'] == stop]
    outgoing = connections.loc[connections['start_code'] == stop]
    if outgoing.shape[0] == 0:
        continue
    value = incoming.shape[0] / outgoing.shape[0]
    for out in outgoing.stop_code:
        links_matrix[stop][out] = value

100%|██████████| 202/202 [00:00<00:00, 449.19it/s]


In [69]:
# Check for possible NaNs
np.isnan(np.min(links_matrix))

False

In [70]:
links_matrix = links_matrix.transpose()

In [71]:
eigv = np.ones((max_city_category + 1, 1)) * 1 / max_city_category
eigv

array([[0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.00497512],
       [0.004

In [110]:
err = 0.0000000001

def power_iteration(e, A):
    prod = A.dot(e)
    iteration = 0
    while np.linalg.norm(prod - e) > err:
        e = prod
        prod = A.dot(e)
        if np.isnan(prod).any() or np.isinf(prod).any():
            prod = e
            break
        iteration += 1
    print("Converged in " + str(iteration) + " iterations")
    return prod

In [111]:
ranks = power_iteration(eigv.astype(np.float128),  links_matrix.astype(np.float128))

Converged in 5967 iterations


In [113]:
def normalize_columns(arr):
    rows, cols = arr.shape
    for col in range(cols):
        arr[:,col] /= abs(arr[:,col]).max()
    return arr
        
ranks = normalize_columns(ranks)

array([[0.01905667],
       [0.14031209],
       [0.03614655],
       [0.32269574],
       [0.21929239],
       [0.01626991],
       [0.77970072],
       [0.39931167],
       [0.01504526]], dtype=float128)

In [114]:
coordinates = stops[['lat', 'lng', 'city', 'city_cat']].groupby('city').mean()
coordinates['city_rank'] = ranks[:,0]
coordinates.head()

,lat,lng,city_cat,city_rank
city,,,,
Białystok,53.131274,23.134503,0,0.018676
Bielsko-Biała,49.825946,19.051880,1,0.019057
Bolesławiec County,51.265855,15.565740,2,0.140312
Brzeg County,50.860851,17.466831,3,0.036147
Bydgoszcz,53.122432,18.018419,4,0.322696


In [92]:
ranks[1, 2, 3, 4][0]

IndexError: too many indices for array